In [ ]:
#!/usr/bin/python

# Copyright 2018, Gurobi Optimization, LLC

# This example formulates and solves the following simple MIP model:
#  maximize
#        x +   y + 2 z
#  subject to
#        x + 2 y + 3 z <= 4
#        x +   y       >= 1
#  x, y, z binary

from gurobipy import *

try:

    # Create a new model
    m = Model("mip1")

    # Create variables
    x = m.addVar(vtype=GRB.BINARY, name="x")
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")

    # Set objective
    m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

    # Add constraint: x + y >= 1
    m.addConstr(x + y >= 1, "c1")

    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))

    print('Obj: %g' % m.objVal)

except GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Encountered an attribute error')

In [ ]:
#!/usr/bin/python

# Copyright 2018, Gurobi Optimization, LLC

# This example formulates and solves the following simple MIP model:
#  maximize
#        x +   y + 2 z
#  subject to
#        x + 2 y + 3 z <= 4
#        x +   y       >= 1
#  x, y, z binary

from gurobipy import *

try:

    # Create a new model
    m = Model("Economic Dispatch")

    # Create variables
    x = m.addVar(vtype=GRB.BINARY, name="x")
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")

    # Set objective
    m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

    # Add constraint: x + y >= 1
    m.addConstr(x + y >= 1, "c1")

    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))

    print('Obj: %g' % m.objVal)

except GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Encountered an attribute error')

In [ ]:
#!/usr/bin/python

# Copyright 2018, Gurobi Optimization, LLC

# Solve a multi-commodity flow problem.  Two products ('Pencils' and 'Pens')
# are produced in 2 cities ('Detroit' and 'Denver') and must be sent to
# warehouses in 3 cities ('Boston', 'New York', and 'Seattle') to
# satisfy demand ('inflow[h,i]').
#
# Flows on the transportation network must respect arc capacity constraints
# ('capacity[i,j]'). The objective is to minimize the sum of the arc
# transportation costs ('cost[i,j]').

from gurobipy import *

# Model data

commodities = ['Pencils', 'Pens']
nodes = ['Detroit', 'Denver', 'Boston', 'New York', 'Seattle']

arcs, capacity = multidict({
  ('Detroit', 'Boston'):   100,
  ('Detroit', 'New York'):  80,
  ('Detroit', 'Seattle'):  120,
  ('Denver',  'Boston'):   120,
  ('Denver',  'New York'): 120,
  ('Denver',  'Seattle'):  120 })

cost = {
  ('Pencils', 'Detroit', 'Boston'):   10,
  ('Pencils', 'Detroit', 'New York'): 20,
  ('Pencils', 'Detroit', 'Seattle'):  60,
  ('Pencils', 'Denver',  'Boston'):   40,
  ('Pencils', 'Denver',  'New York'): 40,
  ('Pencils', 'Denver',  'Seattle'):  30,
  ('Pens',    'Detroit', 'Boston'):   20,
  ('Pens',    'Detroit', 'New York'): 20,
  ('Pens',    'Detroit', 'Seattle'):  80,
  ('Pens',    'Denver',  'Boston'):   60,
  ('Pens',    'Denver',  'New York'): 70,
  ('Pens',    'Denver',  'Seattle'):  30 }

inflow = {
  ('Pencils', 'Detroit'):   50,
  ('Pencils', 'Denver'):    60,
  ('Pencils', 'Boston'):   -50,
  ('Pencils', 'New York'): -50,
  ('Pencils', 'Seattle'):  -10,
  ('Pens',    'Detroit'):   60,
  ('Pens',    'Denver'):    40,
  ('Pens',    'Boston'):   -40,
  ('Pens',    'New York'): -30,
  ('Pens',    'Seattle'):  -30 }

# Create optimization model
m = Model('netflow')

# Create variables
flow = m.addVars(commodities, arcs, obj=cost, name="flow")

# Arc capacity constraints
m.addConstrs(
    (flow.sum('*',i,j) <= capacity[i,j] for i,j in arcs), "cap")

# Equivalent version using Python looping
# for i,j in arcs:
#   m.addConstr(sum(flow[h,i,j] for h in commodities) <= capacity[i,j],
#               "cap[%s,%s]" % (i, j))


# Flow conservation constraints
m.addConstrs(
    (flow.sum(h,'*',j) + inflow[h,j] == flow.sum(h,j,'*')
    for h in commodities for j in nodes), "node")
# Alternate version:
# m.addConstrs(
#   (quicksum(flow[h,i,j] for i,j in arcs.select('*',j)) + inflow[h,j] ==
#     quicksum(flow[h,j,k] for j,k in arcs.select(j,'*'))
#     for h in commodities for j in nodes), "node")

# Compute optimal solution
m.optimize()

# Print solution
if m.status == GRB.Status.OPTIMAL:
    solution = m.getAttr('x', flow)
    for h in commodities:
        print('\nOptimal flows for %s:' % h)
        for i,j in arcs:
            if solution[h,i,j] > 0:
                print('%s -> %s: %g' % (i, j, solution[h,i,j]))

In [ ]:
Generators = ['Gen1','Gen2','Gen3']
Bus = ['Bus1', 'Bus2', 'Bus3']

lines, Susceptance = multidict({
  ('Bus1', 'Bus1'): 18,
  ('Bus1', 'Bus2'): -10,
  ('Bus1', 'Bus3'): -8,
  ('Bus2',  'Bus1'): -10,
  ('Bus2',  'Bus2'): 15,
  ('Bus2',  'Bus3'): -5,
  ('Bus3',  'Bus1'): -8,
  ('Bus3',  'Bus2'): -5,
  ('Bus3',  'Bus3'):  13,})

In [ ]:
B= [100*Susceptance[(i,j)] for i,j in lines]

In [ ]:
Gen = {'Gen1':{'a':0.00562 ,'b': 7.92,'c': 561, 'Pmin': 150,'Pmax': 600},
      'Gen2':{'a':0.00194 ,'b': 7.85,'c': 310, 'Pmin': 100,'Pmax': 400},
      'Gen3':{'a':0.00482 ,'b': 7.97,'c': 78, 'Pmin': 50,'Pmax': 200}}
Gen

In [ ]:
Gen['Gen1']['a']

In [2]:
Generators =['Gen1', 'Gen2', 'Gen3']
GenAtri = {'Gen1':{'a':0.00562 ,'b': 7.92,'c': 561, 'Pmin': 150,'Pmax': 600},
           'Gen2':{'a':0.00194 ,'b': 7.85,'c': 310, 'Pmin': 100,'Pmax': 400},
           'Gen3':{'a':0.00482 ,'b': 7.97,'c': 78, 'Pmin': 50,'Pmax': 200}}
Gen

P=1

GenCost = [Gen[i]['c']+Gen[i]['b']*P+Gen[i]['a']*P**2 for i in Gen]
GenCost

[568.92562, 317.85194, 85.97482]

In [6]:
P=1
Generators = ['Gen1','Gen2', 'Gen3']


#Set Objective
GenCost=[Gen[i]['c']+Gen[i]['b']*P+Gen[i]['a']*P**2 for i in Gen]
Pi[1]           

'Gen2'

In [7]:


Obj = [Gen[i]['c']+Gen[i]['b']*P[i]+Gen[i]['a']P[i]**2 
           for i in Gen]

Demand = 850

sum(P[i])=Demand


SyntaxError: invalid syntax (<ipython-input-7-dc66bcb6274a>, line 3)

In [26]:
from gurobipy import *
EC = Model('Economic Dispatch')
Gen = ['Gen1','Gen2', 'Gen3']
GenAt = {Gen[0]:{'a':0.00562 ,'b': 7.92,'c': 561, 'Pmin': 150,'Pmax': 600},
      Gen[1]:{'a':0.00194 ,'b': 7.85,'c': 310, 'Pmin': 100,'Pmax': 400},
      Gen[2]:{'a':0.00482 ,'b': 7.97,'c': 78, 'Pmin': 50,'Pmax': 200}}
P = EC.addVars(Gen, name = 'Power')
Demand = 850
GenTotal = quicksum(P[i] for i in Gen)
GenCost = quicksum(GenAt[i]['c']+GenAt[i]['b']*P[i]+GenAt[i]['a']*P[i]*P[i] for i in Gen)

EC.setObjective(GenCost, GRB.MINIMIZE)

DemandConst = EC.addConstr( GenTotal == Demand, name = 'DemandConst')
Pmax = EC.addConstrs((P[i] <= GenAt[i]['Pmax'] for i in Gen), name='Maximum Generation Constraint')
Pmin = EC.addConstrs((P[i] >= GenAt[i]['Pmin'] for i in Gen), name='Minimum Generation Constraint')
EC.optimize()

for v in EC.getVars():
    print(v.varName, v.x)

print('Obj:', EC.objVal)
print('λ:',DemandConst.getAttr('PI'))


Optimize a model with 7 rows, 3 columns and 9 nonzeros
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 8e+00]
  QObjective range [4e-03, 1e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 9e+02]
Presolve removed 6 rows and 0 columns
Presolve time: 0.02s
Presolved: 1 rows, 3 columns, 3 nonzeros
Presolved model has 3 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.49214198e+06 -1.29294756e+06  2.00e+03 9.93e+02  1.02e+06     0s
   1   8.80212939e+03 -8.20226420e+04  2.00e-03 9.93e-04  1.51e+04     0s
   2   8.78912307e+03  8.09572566e+03  7.19e-06 3.57e-06  1.16e+02     0s
   3   8.54323377e+03  8.44387586e+03  7.18e-12 3.5

In [73]:
from gurobipy import *
method =1
EC = Model('Economic Dispatch')
Gen = ['Gen1','Gen2', 'Gen3', 'Gen4', 'Gen5']
GenAt = {Gen[0]:{'a':3 ,'b':20 ,'c':100 , 'Pmin':28 ,'Pmax':206 },
      Gen[1]:{'a':4.05 ,'b':18.07 ,'c':98.87 , 'Pmin':90 ,'Pmax':284 },
      Gen[2]:{'a':4.05 ,'b':15.55 ,'c':104.26 , 'Pmin':68 ,'Pmax':189 },
      Gen[3]:{'a':3.99 ,'b':19.21 ,'c':107.21 , 'Pmin':76 ,'Pmax':266 },  
      Gen[4]:{'a':3.88 ,'b':26.18 ,'c':95.31 , 'Pmin':19 ,'Pmax':53 }  }
P = EC.addVars(Gen, name = 'Power')
Demand = 400
GenTotal = quicksum(P[i] for i in Gen)
GenCost = quicksum(GenAt[i]['c']+GenAt[i]['b']*P[i]+GenAt[i]['a']*P[i]*P[i] for i in Gen)

EC.setObjective(GenCost, GRB.MINIMIZE)

DemandConst = EC.addConstr( GenTotal == Demand, name = 'DemandConst')
Pmax = EC.addConstrs((P[i] <= GenAt[i]['Pmax'] for i in Gen), name='Maximum Generation Constraint')
Pmin = EC.addConstrs((P[i] >= GenAt[i]['Pmin'] for i in Gen), name='Minimum Generation Constraint')


#Algorithm used to solve continuous models or the root node of a MIP model. 
#Options are: -1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 
#              3=concurrent, 4=deterministic concurrent, 5=deterministic concurrent simplex.
EC.Params.Method=5

EC.optimize()

for v in EC.getVars():
    print(v.varName, v.x)

print('Obj:', EC.objVal)
print('λ:',DemandConst.getAttr('PI'))



for i in range(len(Gen)):
    print('μ for '+ str(Gen[i]),Pmax[Gen[i]].getAttr('PI'))

Changed value of parameter Method to 5
   Prev: -1  Min: -1  Max: 5  Default: -1
Optimize a model with 11 rows, 5 columns and 15 nonzeros
Model has 5 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 3e+01]
  QObjective range [6e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+02]
         Using dual simplex
Presolve removed 10 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 5 columns, 5 nonzeros
Presolved model has 5 quadratic objective terms

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4037850e+04   0.000000e+00   0.000000e+00      0s
       6    1.3145500e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.02 seconds
Optimal objective  1.314550003e+05
Power[Gen1] 102.84422597102746
Power[Gen2] 90.0
Power[Gen3] 76.73029084273637
Power[Gen4] 77.42548318623614
Power[Gen5] 53.0
Obj: 131455.00026067847
λ: 637.0653558261645
μ for Gen1 0.0

In [62]:
from gurobipy import *
from prettytable import PrettyTable 
method =1
EC = Model('Economic Dispatch')
Gen = ['Gen1','Gen2', 'Gen3', 'Gen4', 'Gen5']
GenAt = {Gen[0]:{'a':3 ,'b':20 ,'c':100 , 'Pmin':28 ,'Pmax':206 },
      Gen[1]:{'a':4.05 ,'b':18.07 ,'c':98.87 , 'Pmin':90 ,'Pmax':284 },
      Gen[2]:{'a':4.05 ,'b':15.55 ,'c':104.26 , 'Pmin':68 ,'Pmax':189 },
      Gen[3]:{'a':3.99 ,'b':19.21 ,'c':107.21 , 'Pmin':76 ,'Pmax':266 },  
      Gen[4]:{'a':3.88 ,'b':26.18 ,'c':95.31 , 'Pmin':19 ,'Pmax':53 }  }
P = EC.addVars(Gen,lb = [GenAt[i]['Pmin'] for i in Gen], ub = [GenAt[i]['Pmax'] for i in Gen],name = 'Power')

Demand = 281
GenTotal = quicksum(P[i] for i in Gen)
GenCost = quicksum(GenAt[i]['c']+GenAt[i]['b']*P[i]+GenAt[i]['a']*P[i]*P[i] for i in Gen)
EC.setObjective(GenCost, GRB.MINIMIZE)
DemandConst = EC.addConstr( GenTotal == Demand, name = 'DemandConst')

#Algorithm used to solve continuous models or the root node of a MIP model. 
#Options are: -1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 
#              3=concurrent, 4=deterministic concurrent, 5=deterministic concurrent simplex.
EC.Params.Method=5

EC.optimize()

print('Total Cost:', "€",round(EC.objVal,2))
print('Cost of Next MW:', "€",round(DemandConst.getAttr('PI'),2))

Results = PrettyTable(['Generator','Lower (MW)','Value (MW)','Upper (MW)', 'Marginal (€)'])
for v in EC.getVars():
    Results.add_row([v.varName, v.LB,round((v.x),3), v.UB,round(v.getAttr('RC'),3)])
print(Results)
    

Changed value of parameter Method to 5
   Prev: -1  Min: -1  Max: 5  Default: -1
Optimize a model with 1 rows, 5 columns and 5 nonzeros
Model has 5 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 3e+01]
  QObjective range [6e+00, 8e+00]
  Bounds range     [2e+01, 3e+02]
  RHS range        [3e+02, 3e+02]
         Using dual simplex
Presolve removed 1 rows and 5 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4037850e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  8.403785000e+04
Total Cost: € 84037.85
Cost of Next MW: € 0.0
+-------------+------------+------------+------------+--------------+
|  Generator  | Lower (MW) | Value (MW) | Upper (MW) | Marginal (€) |
+-------------+------------+------------+------------+--------------+
| Power[Gen1] |    28.0    |    28.0    |   20

In [56]:
from gurobipy import *
from prettytable import PrettyTable 
method =1
EC = Model('Economic Dispatch')
Gen = ['Gen1','Gen2', 'Gen3', 'Gen4', 'Gen5']
GenAt = {Gen[0]:{'a':3 ,'b':20 ,'c':100 , 'Pmin':28 ,'Pmax':206 },
      Gen[1]:{'a':4.05 ,'b':18.07 ,'c':98.87 , 'Pmin':90 ,'Pmax':284 },
      Gen[2]:{'a':4.05 ,'b':15.55 ,'c':104.26 , 'Pmin':68 ,'Pmax':189 },
      Gen[3]:{'a':3.99 ,'b':19.21 ,'c':107.21 , 'Pmin':76 ,'Pmax':266 },  
      Gen[4]:{'a':3.88 ,'b':26.18 ,'c':95.31 , 'Pmin':19 ,'Pmax':53 }  }
P = EC.addVars(Gen,lb = [GenAt[i]['Pmin'] for i in Gen], ub = [GenAt[i]['Pmax'] for i in Gen],name = 'Power')

Demand = 400
GenTotal = quicksum(P[i] for i in Gen)
GenCost = quicksum(GenAt[i]['c']+GenAt[i]['b']*P[i]+GenAt[i]['a']*P[i]*P[i] for i in Gen)
EC.setObjective(GenCost, GRB.MINIMIZE)
DemandConst = EC.addConstr( GenTotal == Demand, name = 'DemandConst')

#Algorithm used to solve continuous models or the root node of a MIP model. 
#Options are: -1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 
#              3=concurrent, 4=deterministic concurrent, 5=deterministic concurrent simplex.
EC.Params.Method=0

EC.optimize()

print('Total Cost:', "€",round(EC.objVal,2))
print('Cost of Next MW:', "€",round(DemandConst.getAttr('PI'),2))

Results = PrettyTable(['Generator','Lower (MW)','Value (MW)','Upper (MW)', 'Marginal (€)'])
for v in EC.getVars():
    Results.add_row([v.varName, v.LB,round((v.x),3), v.UB,round(v.getAttr('RC'),3)])
print(Results)


Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Optimize a model with 1 rows, 5 columns and 5 nonzeros
Model has 5 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 3e+01]
  QObjective range [6e+00, 8e+00]
  Bounds range     [2e+01, 3e+02]
  RHS range        [4e+02, 4e+02]
Presolve time: 0.02s
Presolved: 1 rows, 5 columns, 5 nonzeros
Presolved model has 5 quadratic objective terms

       0    1.4889285e+05   0.000000e+00   3.738175e+02      0s
       4    1.3145500e+05   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.03 seconds
Optimal objective  1.314550003e+05
Total Cost: € 131455.0
Cost of Next MW: € 637.07
+-------------+------------+------------+------------+--------------+
|  Generator  | Lower (MW) | Value (MW) | Upper (MW) | Marginal (€) |
+-------------+------------+------------+------------+--------------+
| Power[Gen1] |    28.0    |  102.844   |   206.0    |    

AttributeError: b"Unable to retrieve attribute 'objVal'"

In [59]:
Load =[]
for c in Counter:
    Demand = quicksum(GenAt[i]['Pmin'] for i in Gen) + \
    ((c)/(len(Counter)-1))*quicksum(GenAt[i]['Pmax'] - GenAt[i]['Pmin'] for i in Gen)
    Load.append(Demand)
print(Load)

[<gurobi.LinExpr: 281.0>, <gurobi.LinExpr: 460.25>, <gurobi.LinExpr: 639.5>, <gurobi.LinExpr: 818.75>, <gurobi.LinExpr: 998.0>]


In [52]:
quicksum(GenAt[i]['Pmin'] for i in Gen)

<gurobi.LinExpr: 281.0>

In [64]:
((0)/(len(Counter)-1))

0.0

In [65]:
from gurobipy import *
from prettytable import PrettyTable 
method =1
EC = Model('Economic Dispatch')
Gen = ['Gen1','Gen2', 'Gen3', 'Gen4', 'Gen5']
GenAt = {Gen[0]:{'a':3 ,'b':20 ,'c':100 , 'Pmin':28 ,'Pmax':206 },
      Gen[1]:{'a':4.05 ,'b':18.07 ,'c':98.87 , 'Pmin':90 ,'Pmax':284 },
      Gen[2]:{'a':4.05 ,'b':15.55 ,'c':104.26 , 'Pmin':68 ,'Pmax':189 },
      Gen[3]:{'a':3.99 ,'b':19.21 ,'c':107.21 , 'Pmin':76 ,'Pmax':266 },  
      Gen[4]:{'a':3.88 ,'b':26.18 ,'c':95.31 , 'Pmin':19 ,'Pmax':53 }  }
P = EC.addVars(Gen,lb = [GenAt[i]['Pmin'] for i in Gen], ub = [GenAt[i]['Pmax'] for i in Gen],name = 'Power')

Counter = range(5)
List =[]
for c in Counter:
    
    Demand = quicksum(GenAt[i]['Pmin'] for i in Gen) + \
    ((c)/(len(Counter)-1))*quicksum(GenAt[i]['Pmax'] - GenAt[i]['Pmin'] for i in Gen)
    
    GenTotal = quicksum(P[i] for i in Gen)
    GenCost = quicksum(GenAt[i]['c']+GenAt[i]['b']*P[i]+GenAt[i]['a']*P[i]*P[i] for i in Gen)
    EC.setObjective(GenCost, GRB.MINIMIZE)
    DemandConst = EC.addConstr( GenTotal == Demand, name = 'DemandConst')

    EC.Params.Method=0
    EC.optimize()
    
    print('Total Cost:', "€",round(EC.objVal,2))
    print('Cost of Next MW:', "€",round(DemandConst.getAttr('PI'),2))

Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Optimize a model with 1 rows, 5 columns and 5 nonzeros
Model has 5 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 3e+01]
  QObjective range [6e+00, 8e+00]
  Bounds range     [2e+01, 3e+02]
  RHS range        [3e+02, 3e+02]
Presolve removed 1 rows and 5 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.4037850e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  8.403785000e+04
Total Cost: € 84037.85
Cost of Next MW: € 0.0
Parameter Method unchanged
   Value: 0  Min: -1  Max: 5  Default: -1
Optimize a model with 2 rows, 5 columns and 10 nonzeros
Model has 5 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 3e+01]
  QObjective range [6e+00, 8e

AttributeError: b"Unable to retrieve attribute 'objVal'"